In [2783]:
%run data_plot.ipynb

In [2784]:
dga_data = {'best_path': {},
            'possible_paths': {}}
solutions = [{}]

In [2785]:
def dga_init_population(origin, destination, pop_len):
    """
    init_population(origin, destination, pop_len)
    initializes a population that represents various permutations of possible paths
    from the origin node to destination node. However due to stochastic nature of the 
    initialization, some paths are invalid while others are possibly repeated. 
    This problem will be handled in 'get_valid_paths()' function.
    origin: the node from which you want to start the search
    destination: the node to which you are trying to establish the best path (highest bandwidth)
    returns: various permutations of the paths with origin node at the start and destination
    node at the end
    """
    routes = []
    '''
    get indices that can be randomly used to create a population but 
    exclude the origin and destination since they will have a fixed position
    '''
    node_indices = [loc for loc in x_y_data.keys() if loc != origin and loc != destination]
    # generate population of length defined as a parameter
    for i in range(pop_len):
        # length of path (chromosome) is not fixed so generate a random length of path for each iteration
        path_length = randint(1, len(node_indices))
        # choose indices to be used as genes
        chosen_indices = choice(np.array(node_indices), path_length, replace=False)
        # prepend and append origin and destination respectively to the generated path
        route = np.append(origin, chosen_indices)
        route = np.append(route, destination)
        # add route to list of routes (the population)
        routes.append(route)

    return routes

In [2786]:
#routes = dga_init_population(52, 100, 100000)
#routes

In [2787]:
def bandwidth_latency_calc(routes):
    """
    bandwidth_calc(routes)
    calculates the end-to-end transmission rate for each route provided
    (represented as a chromosome)
    routes: the population (list of paths/chromosomes) whose end-to-end transmissions
    are being calculated
    returns: a list of tuples containing chromosomes and their corresponding end-to-end
    transmission cost
    """
    global dga_data, solutions
    route_costs = []
    # loop through every route and get the complete end-to-end transmission rate
    for route in routes:
        # var for storing all path bandwidths
        path_bandwidths = []
        path_latency = 0
        # we do -1 because we need to get to second last node where we will check cost between it and last node
        for i in range(len(route) - 1):
            # get path distance
            path_distance = calculate_distance(x_y_data[route[i]], x_y_data[route[i + 1]])
            # get bandwidth between node and its next node and append it to path bandwidths var
            path_bandwidth = convert_distance_bandwidth(path_distance)
            path_latency += node_to_node_latency
            path_bandwidths.append(path_bandwidth)
        # compute end-to-end bandwidth rate
        end_to_end_bandwidth = min(path_bandwidths)
        # step latency down to 0 if bandwidth is 0
        if end_to_end_bandwidth == 0:
            absolute_path_cost = 0
        else:
            absolute_path_cost = round(absolute_cost_function(end_to_end_bandwidth, path_latency), 3)
            if len(dga_data['best_path']) == 0:
                solutions[0] = {"source node": f"Node-{route[0]}"}
                routing_path = ''
                for idx, sensor in enumerate(route[1:]):
                    routing_path += f"(Node-{sensor}, {path_bandwidths[idx]} Mbps),"
                solutions[0]["routing path"] = routing_path
                solutions[0]["end-to-end transmission rate"] = f"{end_to_end_bandwidth} Mbps"
                solutions[0]["end-to-end latency"] = f"{path_latency} ms"
                solutions[0]["absolute path cost"] = f"{absolute_path_cost}"
                dga_data['best_path'][str(list(route))] = {'end_to_end_bandwidth': end_to_end_bandwidth,
                                                     'path_latency': path_latency,
                                                     'absolute_cost': absolute_path_cost
                                                    }
            else:
                best_path = dga_data['best_path'][list(dga_data['best_path'].keys())[0]]
                if absolute_path_cost > best_path['absolute_cost']:
                    solutions[0] = {"source node": f"Node-{route[0]}"}
                    routing_path = ''
                    for idx, sensor in enumerate(route[1:]):
                        routing_path += f"(Node-{sensor}, {path_bandwidths[idx]} Mbps),"
                    solutions[0]["routing path"] = routing_path
                    solutions[0]["end-to-end transmission rate"] = f"{end_to_end_bandwidth} Mbps"
                    solutions[0]["end-to-end latency"] = f"{path_latency} ms"
                    solutions[0]["absolute path cost"] = f"{absolute_path_cost}"
                    dga_data['possible_paths'][list(dga_data['best_path'].keys())[0]] = {
                                                     'end_to_end_bandwidth': best_path['end_to_end_bandwidth'],
                                                     'path_latency': best_path['path_latency'],
                                                     'absolute_cost': best_path['absolute_cost']
                                                    }
                    dga_data['best_path'].pop(list(dga_data['best_path'].keys())[0])
                    dga_data['best_path'][str(list(route))] = {
                                                     'end_to_end_bandwidth': end_to_end_bandwidth,
                                                     'path_latency': path_latency,
                                                     'absolute_cost': absolute_path_cost
                                                    }
                else:
                    dga_data['possible_paths'][str(list(route))] = {
                                                     'end_to_end_bandwidth': end_to_end_bandwidth,
                                                     'path_latency': path_latency,
                                                     'absolute_cost': absolute_path_cost
                                                    }
        # add end-to-end bandwidth to list containing all cost data
        route_costs.append(absolute_path_cost)
    # combine routes and path bandwidths to tuples
    pop_fitness_data = pop_fitness_combine(routes, route_costs)

    return pop_fitness_data

In [2788]:
def pop_fitness_combine(population, fitness_vals):
    """
    pop_fitness_combine(population, fitness_vals)
    combines the population and fitness data into a list
    of tuples that are like (chromosome, fitness val)
    population: list of chromosomes (indices of paths from origin node to 
    destination node)
    fitness_vals: list of fitness values which are ordered in same manner as 
    corresponding chromosomes
    returns: a list of tuples with the chromosome fitness value pair
    """
    pop_fitness_data = []
    for i in range(len(population)):
        pop_fitness_data.append((population[i], fitness_vals[i]))

    return pop_fitness_data

In [2789]:
def get_valid_paths(pop_performance):
    """
    get_valid_paths(pop_performance)
    evaluates the initialized paths and excludes invalid paths on the basis 
    of their end-to-end transmission rates. As defined in the routing table, 
    any none existent node links set to have a bandwidth cost of -infinity. In
    the same function, any end-to-end trnasmission rate that will read -infinity
    will be eliminated. The function further removes duplicated paths for purposes
    of fast convergence and reduction of computational complexities
    pop_performance: list of tuples containing chromosomes and their corresponding end-to-end
    transmission cost
    returns: valid paths with no redundancies (in the same format as their input: list of 
    tuples containing chromosomes and their corresponding end-to-end transmission cost)
    """
    # Remove repeating tuples
    pop_performance = list({tuple(pop): (pop, perform) for pop, perform in pop_performance}.values())
    # init list that will contain only valid paths
    pop_fitness_data = []
    print("********************************************Performance**********************************")
    for individual in  pop_performance:
        # check validity on basis of end-to-end transmission rate. any -infinity denotes an invalid path
        if individual[1] > 0.0:
            path = ''
            '''
            the following lines are for parsing the valid paths into human 
            readable strings for purposes of convergence analysis
            '''
            dest_node_idx = len(individual[0]) - 1
            for idx in individual[0]:
                path += str(idx)
                if idx != dest_node_idx:
                    path += "->"
            print(f"Chromosome: {individual[0]}-----actual route: {path}----Score (end-to-end path cost): {round(individual[1],3)}")
            # finally add valid path to the pop_fitness_data list which will be returned
            pop_fitness_data.append(individual)
    print("*****************************************************************************************")
    if len(pop_fitness_data) == 0:
        print(f"No Solution Found to base station ID {pop_performance[0][0][-1]}")
        #sys.exit("No Solution Found")
    return pop_fitness_data

In [2790]:
#pop_performance = bandwidth_latency_calc(routes)
#pop_fitness_data = get_valid_paths(pop_performance)

In [2791]:
def tournament_fitness_selection(pop_fitness_data, selection_limit=20):
    """
    tournament_fitness_selection(pop_fitness_data, min_selection=10, selection_limit=50)
    fitness selection method that selects the best performing chromosomes (routes) based
    on their end-to-end transmission rates. Paths with higher rates get selected
    pop_fitness_data: list of tuples containing chromosomes and their corresponding 
    end-to-end transmission cost. This list only contains valid paths and has no redundancies
    selection_limit (optional): the maximum number of selected winners beyond which the top n 
    are selected. The default is set to 20
    returns: a list of selected winners (chromosomes)
    """
    comparison_pairs = []
    selected_winners = []
    # loop through each route and generate a random pair of indices that are used for tournament elimination
    if (len(pop_fitness_data) > 1):
        for i in range(len(pop_fitness_data)):
            '''
            generate random pair of indices on basis of length of population. 
            Ensure you exclude the current index to avoid pairing a chromosome to itself
            '''
            possible_pair_indices = [j for j in range(len(pop_fitness_data)) if j != i]
            # append the choices to a tuple
            comparison_pairs.append((i, choice(possible_pair_indices)))
        for j in comparison_pairs:
            # compare the performance data. One with greatest end-to-end cost wins
            if pop_fitness_data[j[0]][1] >= pop_fitness_data[j[1]][1]:
                selected_winners.append(pop_fitness_data[j[0]][0])
        # if selected winners exceeds the selection limit, truncate the list to the limit
        if len(selected_winners) > selection_limit:
            selected_winners = selected_winners[:selection_limit]
    else:
        selected_winners.append(pop_fitness_data[0][0])

    return selected_winners

In [2792]:
#selection_winners = tournament_fitness_selection(pop_fitness_data, selection_limit=20)
#selection_winners

In [2793]:
def crossover_parent_pairing(selection_winners):
    """
    crossover_parent_pairing(selection_winners)
    pairs winners of a selection into parents in readiness for crossover
    selection_winners: the selected winners from based on the evaluation 
    and selection function used
    returns: a 2D array of parents that will be crossed over
    """
    return [selection_winners[i:i+2] for i in range(0, len(selection_winners), 2)]

In [2794]:
#parents = crossover_parent_pairing(selection_winners)
#parents

In [2795]:
def crossover(parents_list, type='single-point'):
    """
    crossover(parents_list, type='single-point')
    function that performs a crossover of 2 parents in the population
    and returns 2 children
    The type of crossover is dependent on the argument passed in as an
    argument. Crossover is done by breaking down the chromosome at randomly
    selected points and concatenating with the other parents genes from that
    position. This is done without changing the genes of the parents since new
    children chromosomes are created.
    parents_list: a list of 2 parents that need a crossover done on them
    type(optional): the type of crossover desired to be done. The default type 
    is single point
    returns: a list of 2 children that result from the crossover
    """
    children_list = []

    if type == 'single-point':
        # if the type is set to single-point, perform a single-point crossover
        crossover_point = randint(1, len(parents_list[0])-1)
        '''
        slice the list from beginning to randomly selected index,
        append slice of second list from that point to the end to the first.
        perform the inverse on the second parent
        '''
        children_list.append([*parents_list[0][:crossover_point],\
                                  *parents_list[1][crossover_point:]])
        children_list.append([*parents_list[1][:crossover_point],\
                                  *parents_list[0][crossover_point:]])
    elif type == 'two-point':
        # if the type is set to two-point, perform a two-point crossover
        crossover_point = randint(low=1, high=len(parents_list[0])-1, size=2)
        '''
        slice the list from beginning to the first randomly selected index, insert 
        part of the second part from that index up until the second randomly selected 
        index and add the last part of the forst index from that point to the end
        perform the inverse on the second parent
        '''
        children_list.append([*parents_list[0][:crossover_point[0]],\
                                  *parents_list[1][crossover_point[0]:crossover_point[1]],\
                                  *parents_list[0][crossover_point[1]:]])
        children_list.append([*parents_list[1][:crossover_point[0]],\
                                  *parents_list[0][crossover_point[0]:crossover_point[1]],\
                                  *parents_list[1][crossover_point[1]:]])
    elif type == 'uniform':
        # if the type is set to uniform, perform uniform crossover
        child1 = []
        child2 = []
        # randomly and uniformly swap bits between the two parents
        for i in range(len(parents_list[0])):
            crossover_point = randint(0, 2)
            if crossover_point == 1:
                child1.append(parents_list[1][i])
                child2.append(parents_list[0][i])
            else:
                child1.append(parents_list[0][i])
                child2.append(parents_list[1][i])
        children_list.append(child1)
        children_list.append(child2)
                
    return np.array(children_list, dtype=object)

In [2796]:
"""
# combining parents and children to form a new generation
children = []
for i in parents:
    if len(i) == 2:
        crossover_result = crossover(i)
        for j in crossover_result:
            children.append(j)
parent_children = np.array(selection_winners + children, dtype=object)
#parent_children
"""

'\n# combining parents and children to form a new generation\nchildren = []\nfor i in parents:\n    if len(i) == 2:\n        crossover_result = crossover(i)\n        for j in crossover_result:\n            children.append(j)\nparent_children = np.array(selection_winners + children, dtype=object)\n#parent_children\n'

In [2797]:
def mutate(population, mutation_rate=0.1, mutation_ratio=0.1):
    """
    mutate(population, mutation_rate=0.1)
    function that performs mutation of random individuals in
    a population. The number of individuals on which mutation
    will take place is determined by the mutation rate 
    population: the population on in which mutation will is performed
    mutation_rate(optional): The percentage of the populations being mutated
    returns: the population that has been mutated based in the parameters provided
    """
    '''
    randomly obtain the indices of the individuals being mutated. 
    The number of indices will approximately be based on the percentage 
    provided as the mutation_rate parameter
    '''
    mutation_chromosome_indices = choice(np.array(range(len(population))),\
                                         int(math.ceil(len(population) * mutation_rate)),\
                                         replace=False)

    #print("Mutation Chromosome indices:"+str(mutation_chromosome_indices))
    # loop through each individual in the population
    for i in mutation_chromosome_indices:
        '''
        randomly obtain the indices of the bits being mutated.
        I have hard coded the 2 to be fair for both long and short paths.
        I have set replace to true because we have occassional small 
        routes selected for mutation whose swap indices are not more than 2.
        Take for example a path [5, 3, 7]; since you cant swap origin and destination
        node '3' cannot be swapped with any other number, therefore it stays as is
        '''
        mutation_gene_indices = choice(np.array(range(len(population[i])))[1:-1],\
                                       2,\
                                         replace=True)
        # swap the genes
        temp = population[i][mutation_gene_indices[0]]
        population[i][mutation_gene_indices[0]] = population[i][mutation_gene_indices[1]]
        population[i][mutation_gene_indices[1]] = temp

    return np.array(population)

In [2798]:
#print("Population before mutation:\n"+str(parent_children))
#print("------------------------------------------")
#print("Population after mutation:\n"+str(mutate(parent_children)))

In [2799]:
def dga_run(origin, destination, pop_length, generations):
    """
    dga_run(origin, destination, pop_length, generations=10)
    consolidates all the sub-functions of the genetic algorithm and
    calls them recurrsively on a generation-to-generation basis as we
    analyse convergence performance
    origin: the origin from which you want to traverse the space
    destination: the destination node which you want to get to
    pop_length: the length of the population for the first generation
    generations (optional): the maximum number of generations that you would 
    want to you do work with (termination criteria). Default is set to 10
    returns: ....
    """
    population = dga_init_population(origin, destination, pop_length)
    for i in range(generations):
        print(f"**********************************Generation {i + 1}**********************************")
        pop_performance = bandwidth_latency_calc(population)
        pop_fitness_data = get_valid_paths(pop_performance)
        selection_winners = tournament_fitness_selection(pop_fitness_data)
        parents = crossover_parent_pairing(selection_winners)
        children = []
        for i in parents:
            if len(i) == 2:
                crossover_result = crossover(i)
                for j in crossover_result:
                    children.append(j)
        parent_children = np.array(selection_winners + children, dtype=object)
        population = mutate(parent_children)
    return dga_data, solutions

In [2800]:
def discrete_genetic_algorithm(origin, pop_length=100000, generations=10):
    """
    discrete_genetic_algorithm(origin, destination, pop_length, generations=10)
    consolidates all the sub-functions of the genetic algorithm and
    calls them recurrsively on a generation-to-generation basis as we
    analyse convergence performance
    origin: the origin from which you want to traverse the space
    destination: the destination node which you want to get to
    pop_length: the length of the population for the first generation
    generations (optional): the maximum number of generations that you would 
    want to you do work with (termination criteria). Default is set to 10
    returns: ...
    """
    try:
        destination = x_y_base_station_1[0]   
        dga_data, solutions = dga_run(origin, destination, pop_length=100000, generations=10)
    except Exception as e:
        solutions = [{}]
        destination = x_y_base_station_2[0]   
        dga_data, solutions = dga_run(origin, destination, pop_length=100000, generations=10)
    return dga_data, solutions

In [2801]:
#dga_results, solutions = discrete_genetic_algorithm(100, 100000)